<a href="https://colab.research.google.com/github/Apyarn95/LanguageTranslationEngToHin/blob/main/LanguageTranlationEngtoHin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from string import digits
import re ,os,io,time
import tensorflow as tf

In [3]:
#mounting the content to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
#loading dataset from drive
data = pd.read_csv('/content/gdrive/My Drive/HindiEnglish.csv',encoding='utf-8')


In [ ]:
new_data = data

In [5]:
#defining contraction mapping to change all the occurences of short form to their original meaning 
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [6]:
#defining preprocessing stuff
def preprocess_sent(w):
  w = w.lower().strip()
  w = ' '.join(contraction_mapping[t] if t in contraction_mapping else t for t in w.split(" "))
  
  #removing everything within brackets
  w = re.sub("([\(\[]).*?([\)\]])", "", w)
  #creating space between word and punctuation following
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  #replace everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = ' '.join(t for t in w.split(" ") if t != " ")
  w = w.strip()
  
  #inclusind <sostoken> and <eostoken> to identify starting and ending of the sentences
  w = '<sostoken> ' + w + ' <eostoken>'
  return w

def preprocess_hindi_sent(w):
  w = w.strip()
  remove_num = str.maketrans('','',digits)
  w = w.translate(remove_num)
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub("([\(\[]).*?([\)\]])", "", w)
  w = w.strip()
  
  w = '<sostoken> ' + w + ' <eostoken>' 
  return w


In [8]:
new_data = data

In [9]:
for x in range(len(new_data)):
  if x not in new_data['english_sentence'].keys():
    continue
  new_data['english_sentence'][x] = preprocess_sent(str(new_data['english_sentence'][x]))
  new_data['hindi_sentence'][x] = preprocess_hindi_sent(str(new_data['hindi_sentence'][x]))


In [10]:
#calculating lenght of the sentences
new_data['eng_len'] = new_data['english_sentence'].apply(lambda x : len(x.split(" ")))
new_data['hin_len'] = new_data['hindi_sentence'].apply(lambda x : len(x.split(" ")))

In [11]:
#removing all intstances where lenght<=25 to reduce dataset for faster computations
sent_max_len = 25
new_data = new_data[new_data['eng_len'] <= sent_max_len]
new_data = new_data[new_data['hin_len'] <= sent_max_len]

In [12]:
#removing null values or duplicates dataset
new_data = new_data[~pd.isnull(data['english_sentence'])]
new_data.drop_duplicates(inplace=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
print(len(new_data))

89282


In [13]:
#converting pandas core series datastructrure to list for further calculations as tensors in tensorflow
english_sentences = new_data['english_sentence'].tolist()
hindi_sentences = new_data['hindi_sentence'].tolist() 

In [14]:

#tokenizing the sentences (changing each sentence to a vector of numbers describing the sentence)
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [15]:
#input_tensor and target_tensor contains vectors representing each sentence where every word has a unique index or token
#to access the word_index we can use inp_lang_tokenizer.word_index where (word) -> (key) type mapping would be there
input_tensor , inp_lang_tokenizer = tokenize(english_sentences)
target_tensor , targ_lang_tokenize = tokenize(hindi_sentences)

In [ ]:
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shap

In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)


In [ ]:
#defining the model runtime attributes
#units -> number of gated GRU cells
#vocab_inp_size is the total size of the english vocab (number of unique words in the corpus) 
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang_tokenizer.word_index)+1
vocab_tar_size = len(targ_lang_tokenize.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
#example_input_batch and example_target batch are the batch iterators that would feed data = batch size from the training to the corpus
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 25]), TensorShape([64, 25]))

In [ ]:
#defining the encoder class
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
#definine the encoder layer with the above defined params
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 25, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
#for attention we would use Bahdanau attention layer for more information read readme.md file
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 25, 1)


In [ ]:
#decoder definition
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 47825)


In [ ]:
#defining optimizers and loss function
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:

checkpoint_prefix = "/content/gdrive/My Drive/LanguageTranslation.ckpt"
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
#training the model
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang_tokenize.word_index['<sostoken>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 20

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
  

Epoch 1 Batch 0 Loss 4.9836
Epoch 1 Batch 100 Loss 3.1134
Epoch 1 Batch 200 Loss 3.6906
Epoch 1 Batch 300 Loss 3.1655
Epoch 1 Batch 400 Loss 3.2391
Epoch 1 Batch 500 Loss 3.3576
Epoch 1 Batch 600 Loss 2.6207
Epoch 1 Batch 700 Loss 3.1969
Epoch 1 Batch 800 Loss 3.0094
Epoch 1 Batch 900 Loss 2.7789
Epoch 1 Batch 1000 Loss 2.8313
Epoch 1 Batch 1100 Loss 2.6783
Epoch 1 Loss 3.1486
Time taken for 1 epoch 494.08093309402466 sec

Epoch 2 Batch 0 Loss 2.9995
Epoch 2 Batch 100 Loss 2.7091
Epoch 2 Batch 200 Loss 2.6454
Epoch 2 Batch 300 Loss 2.6420
Epoch 2 Batch 400 Loss 2.6788
Epoch 2 Batch 500 Loss 2.8468
Epoch 2 Batch 600 Loss 2.4967
Epoch 2 Batch 700 Loss 2.6480
Epoch 2 Batch 800 Loss 2.8598
Epoch 2 Batch 900 Loss 2.6732
Epoch 2 Batch 1000 Loss 2.6031
Epoch 2 Batch 1100 Loss 2.4562
Epoch 2 Loss 2.7003
Time taken for 1 epoch 470.745352268219 sec

Epoch 3 Batch 0 Loss 2.4701
Epoch 3 Batch 100 Loss 2.3665
Epoch 3 Batch 200 Loss 2.5536
Epoch 3 Batch 300 Loss 2.2051
Epoch 3 Batch 400 Loss 2.2951


In [ ]:
# save the model
checkpoint.save(file_prefix = '/content/gdrive/My Drive/Translate.ckpt')

'/content/gdrive/My Drive/Translate.ckpt-12'

In [ ]:
#cconvert the english sentence to hindi using the model
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sent(sentence)

  inputs = [inp_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang_tokenize.word_index['<sostoken>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang_tokenize.index_word[predicted_id] + ' '

    if targ_lang_tokenize.index_word[predicted_id] == '<eostoken>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [ ]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)
  print('Predicted translation: {}'.format(result))


In [ ]:
#example dataset
print('Input : %s ' %(data['english_sentence'][0]))
print('Original translation : %s ' %(data['hindi_sentence'][0]))
print(translate(data['english_sentence'][0]))